Reference: Professor Ami Gates, Dept. Applied Math, Data Science, University of Colorado

[Dr. Gates' Website](https://gatesboltonanalytics.com/?page_id=903)

---

<https://www.tensorflow.org/api_docs/python/tf/keras/Input>

In [1]:
from tensorflow.keras.layers import Activation
import numpy as np
import nltk
import pandas as pd
import sklearn
import re  
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow.keras
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras import layers



from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
## For Stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import os

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

from bs4 import BeautifulSoup
from collections import Counter

from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize



## These settings show/print dfs in diff ways
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 50)
#pd.options.display.max_seq_items = None
#pd.pandas.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 50

# Movies Dataset
<https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format>

(Update the path for YOUR computer)


In [2]:
path = "IMBD_data/"

TrainData = pd.read_csv(str(path+"Train.csv"))
print(TrainData.shape)
print(TrainData.head(10))
print(type(TrainData))

(40000, 2)
                                                text  \
0  I grew up (b. 1965) watching and loving the Th...   
1  When I put this movie in my DVD player, and sa...   
2  Why do people who do not know what a particula...   
3  Even though I have great interest in Biblical ...   
4  Im a die hard Dads Army fan and nothing will e...   
5  A terrible movie as everyone has said. What ma...   
6  Finally watched this shocking movie last night...   
7  I caught this film on AZN on cable. It sounded...   
8  It may be the remake of 1987 Autumn's Tale aft...   
9  My Super Ex Girlfriend turned out to be a plea...   

   label  
0      0  
1      0  
2      0  
3      0  
4      1  
5      0  
6      1  
7      0  
8      1  
9      1  
<class 'pandas.core.frame.DataFrame'>


In [3]:
TestData = pd.read_csv(str(path+"Test.csv"))
print(TestData.shape)
TestData.head(10)

(5000, 2)


,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0
5,"I saw this film on September 1st, 2005 in Indi...",1
6,I saw a screening of this movie last night. I ...,0
7,William Hurt may not be an American matinee id...,1
8,IT IS A PIECE OF CRAP! not funny at all. durin...,0
9,I'M BOUT IT(1997)<br /><br />Developed & publi...,0


In [4]:
ValidData = pd.read_csv(str(path+"Valid.csv"))
print(ValidData.shape)
ValidData.head(10)

(5000, 2)


,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0
5,This is the best movie I've ever seen! <br /><...,1
6,The morbid Catholic writer Gerard Reve (Jeroen...,1
7,"""Semana Santa"" or ""Angel Of Death"" is a very w...",0
8,Somebody mastered the difficult task of mergin...,1
9,Why did I waste 1.5 hours of my life watching ...,0


Concat requires a list

Place all data from above into one dataframe

In [5]:
FullDataset=pd.concat([TrainData,TestData, ValidData])
print(FullDataset.shape)

(50000, 2)


## Clean Up TrainData

Get the vocab

In [6]:
print(TrainData.head())

                                                text  \
0  I grew up (b. 1965) watching and loving the Th...   
1  When I put this movie in my DVD player, and sa...   
2  Why do people who do not know what a particula...   
3  Even though I have great interest in Biblical ...   
4  Im a die hard Dads Army fan and nothing will e...   

   label  
0      0  
1      0  
2      0  
3      0  
4      1  


In [7]:
# Testing iterating the columns 
for col in TrainData.columns: 
    print(col) 

text
label


## Check Content   

In [8]:
print(TrainData["text"])
print(TrainData["label"]) ##0 is negative, 1 is positive

0        I grew up (b. 1965) watching and loving the Th...
1        When I put this movie in my DVD player, and sa...
2        Why do people who do not know what a particula...
3        Even though I have great interest in Biblical ...
4        Im a die hard Dads Army fan and nothing will e...
                               ...                        
39995    "Western Union" is something of a forgotten cl...
39996    This movie is an incredible piece of work. It ...
39997    My wife and I watched this movie because we pl...
39998    When I first watched Flatliners, I was amazed....
39999    Why would this film be so good, but only gross...
Name: text, Length: 40000, dtype: object
0        0
1        0
2        0
3        0
4        1
        ..
39995    1
39996    1
39997    0
39998    1
39999    1
Name: label, Length: 40000, dtype: int64


### Tokenize and Vectorize 

- Create the list 
- Keep the labels

In [9]:
ReviewsLIST=[]  ## from the text column
LabelLIST=[]    

for nextreview, nextlabel in zip(TrainData["text"], TrainData["label"]):
    ReviewsLIST.append(nextreview)
    LabelLIST.append(nextlabel)

print("A Look at some of the reviews list is:\n")
print(ReviewsLIST[0:20])

print("A Look at some of the labels list is:\n")
print(LabelLIST[0:20])

A Look at some of the reviews list is:

['I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.', "When I put this movie in my DVD player, and sat down with a coke and some chips,

## Optional - for Stemming the data


In [10]:
## Instantiate it
A_STEMMER=PorterStemmer()
## test it
print(A_STEMMER.stem("fishers"))

fisher


### Use NLTK's PorterStemmer in a function 

In [11]:
# DEFINE THE FUNCTION
def MY_STEMMER(str_input):
    ## Only use letters, no punct, no nums, make lowercase...
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [A_STEMMER.stem(word) for word in words] ## Use the Stemmer...
    return words

Build the labeled dataframe

Get the Vocab  - here keeping top 10,000

### Vectorize

In [12]:
## Instantiate your CV
MyCountV=CountVectorizer(
        input="content",  
        lowercase=True, 
        #stop_words = "english", ## This is optional
        #tokenizer=MY_STEMMER, ## Stemming is optional
        max_features=11000  ## This can be updated
        )

## Use your CV 
MyDTM = MyCountV.fit_transform(ReviewsLIST)  # create a sparse matrix
print(type(MyDTM))

<class 'scipy.sparse._csr.csr_matrix'>


In [13]:
ColumnNames=MyCountV.get_feature_names_out() ## This is the vocab
print(ColumnNames)

['00' '000' '007' ... 'zoom' 'zorro' 'zucco']


In [14]:
print(type(ColumnNames))

<class 'numpy.ndarray'>


Here we can clean up the columns


In [15]:
## Build the data frame
MyDTM_DF=pd.DataFrame(MyDTM.toarray(),columns=ColumnNames)

## Convert the labels from list to df
Labels_DF = pd.DataFrame(LabelLIST,columns=['LABEL'])

## Check your new DF and you new Labels df:

In [16]:
print("Labels\n")
print(Labels_DF)

Labels

       LABEL
0          0
1          0
2          0
3          0
4          1
...      ...
39995      1
39996      1
39997      0
39998      1
39999      1

[40000 rows x 1 columns]


In [17]:
print("DF\n")
print(MyDTM_DF.iloc[:,0:20])
print(MyDTM_DF.shape) ## 40,000 by 11000

DF

       00  000  007  10  100  1000  101  11  12  \
0       0    0    0   0    0     0    0   0   0   
1       0    0    0   0    0     0    0   0   0   
2       0    0    0   0    0     0    0   0   0   
3       0    0    0   0    0     0    0   0   0   
4       0    0    0   0    0     0    0   0   0   
...    ..  ...  ...  ..  ...   ...  ...  ..  ..   
39995   0    0    0   0    0     0    0   0   0   
39996   0    0    0   0    0     0    0   0   0   
39997   0    0    0   0    0     0    0   0   0   
39998   0    0    0   1    0     0    0   0   0   
39999   0    2    0   0    0     0    0   0   0   

       13  13th  14  15  150  16  17  18  180  \
0       0     0   0   0    0   0   0   0    0   
1       0     0   0   0    0   0   0   0    0   
2       0     0   0   0    0   0   0   0    0   
3       0     0   0   0    0   0   0   0    0   
4       0     0   0   0    0   0   0   0    0   
...    ..   ...  ..  ..  ...  ..  ..  ..  ...   
39995   0     0   0   0    0   0   0   0

Remove any columns that contain numbers

Remove columns with words not the size you want. For example, words < 3 `chars`

## DEFINE A FUNCTION that returns `True` if numbers are in a string 


In [18]:
%%time
def Logical_Numbers_Present(anyString):
    return any(char.isdigit() for char in anyString)
##----------------------------------------------------

for nextcol in MyDTM_DF.columns:
    #print(nextcol)
    ## Remove unwanted columns
    #Result=str.isdigit(nextcol) ## Fast way to check numbers
    #print(Result)
    
    ##-------------call the function -------
    LogResult=Logical_Numbers_Present(nextcol)
    #print(LogResult)
    ## The above returns a logical of True or False
    
    ## The following will remove all columns that contains numbers
    if(LogResult==True):
        #print(LogResult)
        #print(nextcol)
        MyDTM_DF=MyDTM_DF.drop([nextcol], axis=1)

    ## The following will remove any column with name
    ## of 3 or smaller - like "it" or "of" or "pre".
    ##print(len(nextcol))  ## check it first
    ## NOTE: You can also use this code to CONTROL
    ## the words in the columns. For example - you can
    ## have only words between lengths 5 and 9. 
    ## In this case, we remove columns with words <= 3.
    elif(len(str(nextcol))<3):
        #print(nextcol)
        MyDTM_DF=MyDTM_DF.drop([nextcol], axis=1)

CPU times: total: 18min 49s
Wall time: 22min 41s


Save original DF - without the labels

In [19]:
My_Orig_DF=MyDTM_DF
print(My_Orig_DF.head(10))

   aamir  aaron  abandon  abandoned  abbey  \
0      0      0        0          0      0   
1      0      0        0          0      0   
2      0      0        0          0      0   
3      0      0        0          0      0   
4      0      0        0          0      0   
5      0      0        0          0      0   
6      0      0        0          0      0   
7      0      0        0          0      0   
8      0      0        0          0      0   
9      0      0        0          0      0   

   abbott  abc  abduction  abilities  ability  \
0       0    0          0          0        0   
1       0    0          0          0        0   
2       0    0          0          0        0   
3       0    0          0          0        0   
4       0    0          0          0        0   
5       0    0          0          0        0   
6       0    0          0          0        0   
7       0    0          0          0        0   
8       0    0          0          0        0   
9  

Now - let's create a complete and labeled dataframe:

In [23]:
dfs = [Labels_DF, MyDTM_DF]
print(dfs)
print("shape of labels\n", Labels_DF.shape)
print("shape of data\n", MyDTM_DF.shape)

[       LABEL
0          0
1          0
2          0
3          0
4          1
...      ...
39995      1
39996      1
39997      0
39998      1
39999      1

[40000 rows x 1 columns],        aamir  aaron  abandon  abandoned  abbey  \
0          0      0        0          0      0   
1          0      0        0          0      0   
2          0      0        0          0      0   
3          0      0        0          0      0   
4          0      0        0          0      0   
...      ...    ...      ...        ...    ...   
39995      0      0        0          0      0   
39996      0      0        0          0      0   
39997      0      0        0          0      0   
39998      0      0        0          0      0   
39999      0      0        0          0      0   

       abbott  abc  abduction  abilities  \
0           0    0          0          0   
1           0    0          0          0   
2           0    0          0          0   
3           0    0          0          

In [24]:
Final_DF_Labeled = pd.concat(dfs,axis=1, join='inner')
## DF with labels
print(Final_DF_Labeled.iloc[:, 0:2])
print(Final_DF_Labeled.shape)

       LABEL  aamir
0          0      0
1          0      0
2          0      0
3          0      0
4          1      0
...      ...    ...
39995      1      0
39996      1      0
39997      0      0
39998      1      0
39999      1      0

[40000 rows x 2 columns]
(40000, 10712)


FYI

An alternative option for most frequent 10,000 words 

Not needed here as we used `CountVectorizer` with option `max_features`

```
print (df.shape[0])
print (df[:10000].value.sum()/df.value.sum())
top_words = list(df[:10000].key.values)
print(top_words)
## Example using index
index = top_words.index("humiliating")
print(index)
```

In [25]:
## Create list of all words
print(Final_DF_Labeled.columns[0])
NumCols=Final_DF_Labeled.shape[1]
print(NumCols)

LABEL
10712


In [26]:
print(len(list(Final_DF_Labeled.columns)))

10712


In [27]:
top_words=list(Final_DF_Labeled.columns[1:NumCols+1])

In [28]:
## Exclude the Label
print(top_words[0])
print(top_words[-1])

aamir
zucco


In [29]:
print(type(top_words))
print(top_words.index("aamir")) ## index 0 in top_words
print(top_words.index("zucco")) #index NumCols - 2 in top_words

<class 'list'>
0
10710


In [30]:
## Encoding the data
def Encode(review):
    words = review.split()
   # print(words)
    if len(words) > 500:
        words = words[:500]
        #print(words)
    encoding = []
    for word in words:
        try:
            index = top_words.index(word)
        except:
            index = (NumCols - 1)
        encoding.append(index)
    while len(encoding) < 500:
        encoding.append(NumCols)
    return encoding

Test the code to assure that it is doing what you think it should 

In [31]:
result1 = Encode("aaron aamir abbey abbott abilities zucco ")
print(result1)
result2 = Encode("york young younger youngest youngsters youth youthful youtube zach zane zany zealand zellweger")
print(result2)
print(len(result2)) ## Will be 500 because we set it that way above

[1, 0, 4, 5, 8, 10710, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 10712, 1071

Now we are ready to encode all of our reviews - which are called "text" in our dataset. 

Using vocab from above i -  convert reviews (text) into numerical form.
Replacing each word with its corresponding integer index value from the 
vocabulary. Words not in the vocab will
be assigned  as the max length of the vocab + 1 


Encode our training and testing datasets
with same vocab. 


In [32]:
print(TestData.head(10))
print(TestData.shape)
print(TrainData.shape)

                                                text  \
0  I always wrote this series off as being a comp...   
1  1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...   
2  This movie was so poorly written and directed ...   
3  The most interesting thing about Miryang (Secr...   
4  when i first read about "berlin am meer" i did...   
5  I saw this film on September 1st, 2005 in Indi...   
6  I saw a screening of this movie last night. I ...   
7  William Hurt may not be an American matinee id...   
8  IT IS A PIECE OF CRAP! not funny at all. durin...   
9  I'M BOUT IT(1997)<br /><br />Developed & publi...   

   label  
0      0  
1      0  
2      0  
3      1  
4      0  
5      1  
6      0  
7      1  
8      0  
9      0  
(5000, 2)
(40000, 2)


## Final Training and Testing data and labels

In [33]:
%%time
training_data = np.array([Encode(review) for review in TrainData["text"]])
print(training_data[20])
print(training_data.shape)

[10711  8675 10711  4420 10711  9599 10711 10711 10204 10711 10711 10711
 10711  9599 10711 10711 10711 10711  9599  4453 10711 10711 10711 10711
 10711  3646  3822 10711 10711  8268  9645  6251    16  9666 10666   202
  6491  4397   412  4710 10483 10711 10711 10711 10711 10711  5556 10711
 10711 10711 10711  4116  9150 10711 10711  3291  4378 10711  5604  1522
  2624 10473 10711  4189  3773 10711 10711 10711 10711 10711 10678  4026
  1060 10558  2030 10711 10711  4026 10711 10711 10711  6491  8270 10711
  5219 10711  6423 10711 10711 10711  7298  4116  6251 10711 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10

In [34]:
%%time
testing_data = np.array([Encode(review) for review in TestData['text']])
print(testing_data[20])

[10711 10711  1714  7176 10679 10711  5993   343 10579 10711 10711 10711
 10711  7657 10711 10711  3572  9597   259 10711  9599    94   343    96
 10445  5768   343  7666  2649 10711  4189  5150 10711  1359  9599 10711
 10711 10711   343 10711 10711 10599 10439  9741   343 10711  4601 10711
  8388  6208 10598  3884  9599  9965 10711 10711 10711 10711  5917 10711
  3459   259 10711  9599 10711  2649  9273 10711  3655  5150 10711 10622
 10711  4189 10711  8388   393 10711  3884 10711 10711  6251  1320 10711
  1883 10711  6223   412 10711  8101 10711 10711 10678  4395  6491  8400
  9645  6251   285 10711  9194  7695  9597 10678 10711 10711  1320  2471
  4060 10678   393  6952 10711  5535   343 10711 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712 10712
 10712 10712 10712 10712 10712 10712 10712 10712 10

In [35]:
%%time
validation_data = np.array([Encode(review) for review in ValidData['text']])

print (training_data.shape, testing_data.shape)

(40000, 500) (5000, 500)
CPU times: total: 7min 1s
Wall time: 8min 11s


Prepare the labels if they are not already 0 and 1. In our case they are so these lines are commented out and just FYI
```
train_labels = [1 if label=='positive' else 0 for sentiment in TrainData['label']]
test_labels = [1 if label=='positive' else 0 for sentiment in TestData['label']]
```


In [36]:
train_labels = np.array([TrainData['label']])
train_labels=train_labels.T
print(train_labels.shape)
test_labels = np.array([TestData['label']])
test_labels=test_labels.T
print(test_labels.shape)

(40000, 1)
(5000, 1)


# ANN

Simple Dense NN for sentiment analysis (classification 0 neg, 1 pos)

- *First layer*: Embedding Layer (Keras Embedding Layer) that will learn embeddings for different words.
    - **RE:** <https://keras.io/api/layers/core_layers/embedding/>
- `input_dim`: Integer. Size of the vocabulary
- `input_length`: Length of input sequences, when it is constant.


In [37]:
print(NumCols)   
input_dim = NumCols + 1 

10712


In [38]:
My_ANN_Model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=input_dim, output_dim=32, input_length=500),
  tf.keras.layers.Dense(32, activation='relu'), 
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(.5), 
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dropout(.5), 
  tf.keras.layers.Dense(1, activation='sigmoid')
  
])

In [39]:
My_ANN_Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           342816    
                                                                 
 dense (Dense)               (None, 500, 32)           1056      
                                                                 
 dense_1 (Dense)             (None, 500, 16)           528       
                                                                 
 dropout (Dropout)           (None, 500, 16)           0         
                                                                 
 dense_2 (Dense)             (None, 500, 8)            136       
                                                                 
 dropout_1 (Dropout)         (None, 500, 8)            0         
                                                                 
 dense_3 (Dense)             (None, 500, 1)            9

In [40]:
loss_function = keras.losses.BinaryCrossentropy(from_logits=False)
My_ANN_Model.compile(
                 loss=loss_function,
                 metrics=["accuracy"],
                 optimizer='adam'
                 )

In [41]:
print(training_data[0:3, 0:3])
print(training_data.shape)
print(train_labels[10])

[[10711  4207 10711]
 [10711 10711  7496]
 [10711 10711  6912]]
(40000, 500)
[1]


In [42]:
%%time
Hist=My_ANN_Model.fit(training_data, train_labels, epochs=5, validation_data=(testing_data, test_labels))

Epoch 1/5


1250/1250 [==============================] - 44s 33ms/step - loss: 0.6921 - accuracy: 0.5078 - val_loss: 0.6909 - val_accuracy: 0.5157
Epoch 2/5
1250/1250 [==============================] - 39s 31ms/step - loss: 0.6913 - accuracy: 0.5099 - val_loss: 0.6903 - val_accuracy: 0.5169
Epoch 3/5
1250/1250 [==============================] - 39s 31ms/step - loss: 0.6910 - accuracy: 0.5157 - val_loss: 0.6903 - val_accuracy: 0.5166
Epoch 4/5
1250/1250 [==============================] - 43s 34ms/step - loss: 0.6910 - accuracy: 0.5121 - val_loss: 0.6905 - val_accuracy: 0.5097
Epoch 5/5
1250/1250 [==============================] - 42s 34ms/step - loss: 0.6910 - accuracy: 0.5112 - val_loss: 0.6902 - val_accuracy: 0.5171
CPU times: total: 12min 49s
Wall time: 3min 27s


# SimpleRNN


In [43]:
##batch_size=256,
output_dim=32
print(input_dim)
print(training_data.shape)

10713
(40000, 500)


In [44]:
My_SimpleRNN_Model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim, input_length=500),
  tf.keras.layers.SimpleRNN(units =50, input_shape=(32,32,3)),
  ## If not using Embedding, you would use SimpleRNN(units, input_shape=(x,y))
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [45]:
My_SimpleRNN_Model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           342816    
                                                                 
 simple_rnn (SimpleRNN)      (None, 50)                4150      
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 347,017
Trainable params: 347,017
Non-trainable params: 0
_________________________________________________________________


In [46]:
loss_function = keras.losses.BinaryCrossentropy(from_logits=False)
My_SimpleRNN_Model.compile(
                 loss=loss_function,
                 metrics=["accuracy"],
                 optimizer='adam'
                 )

In [47]:
print(training_data[0:3, 0:3])

[[10711  4207 10711]
 [10711 10711  7496]
 [10711 10711  6912]]


In [48]:
print(train_labels[10])

[1]


In [49]:
%%time
Hist=My_SimpleRNN_Model.fit(training_data, train_labels, epochs=5, validation_data=(testing_data, test_labels))

Epoch 1/5


1250/1250 [==============================] - 322s 254ms/step - loss: 0.6957 - accuracy: 0.5042 - val_loss: 0.6944 - val_accuracy: 0.4996
Epoch 2/5
1250/1250 [==============================] - 315s 252ms/step - loss: 0.6939 - accuracy: 0.5048 - val_loss: 0.6935 - val_accuracy: 0.4986
Epoch 3/5
1250/1250 [==============================] - 319s 255ms/step - loss: 0.6926 - accuracy: 0.5087 - val_loss: 0.6959 - val_accuracy: 0.4992
Epoch 4/5
1250/1250 [==============================] - 315s 252ms/step - loss: 0.6898 - accuracy: 0.5146 - val_loss: 0.6943 - val_accuracy: 0.5046
Epoch 5/5
1250/1250 [==============================] - 332s 265ms/step - loss: 0.6846 - accuracy: 0.5196 - val_loss: 0.7058 - val_accuracy: 0.5044
CPU times: total: 1h 42min 34s
Wall time: 26min 43s


## RNN with Bidirectional 


<https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional>


In [50]:
My_RNN_Model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=input_dim, output_dim=32, input_length=500),
  tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(50)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [51]:
My_RNN_Model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           342816    
                                                                 
 bidirectional (Bidirectiona  (None, 100)              8300      
 l)                                                              
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 351,217
Trainable params: 351,217
Non-trainable params: 0
_________________________________________________________________


In [52]:
loss_function = keras.losses.BinaryCrossentropy(from_logits=False)
My_RNN_Model.compile(
                 loss=loss_function,
                 metrics=["accuracy"],
                 optimizer='adam'
                 )

In [53]:
print(training_data[0:3, 0:3])
print(training_data.shape)
print(train_labels[10])

[[10711  4207 10711]
 [10711 10711  7496]
 [10711 10711  6912]]
(40000, 500)
[1]


In [54]:
%%time
Hist=My_RNN_Model.fit(training_data, train_labels, epochs=5, validation_data=(testing_data, test_labels))

Epoch 1/5
1250/1250 [==============================] - 519s 409ms/step - loss: 0.6839 - accuracy: 0.5469 - val_loss: 0.6778 - val_accuracy: 0.5798
Epoch 2/5
1250/1250 [==============================] - 498s 398ms/step - loss: 0.6762 - accuracy: 0.5644 - val_loss: 0.6895 - val_accuracy: 0.5192
Epoch 3/5
1250/1250 [==============================] - 498s 398ms/step - loss: 0.6738 - accuracy: 0.5724 - val_loss: 0.6717 - val_accuracy: 0.5856
Epoch 4/5
1250/1250 [==============================] - 492s 394ms/step - loss: 0.6091 - accuracy: 0.6740 - val_loss: 0.5643 - val_accuracy: 0.7342
Epoch 5/5
1250/1250 [==============================] - 491s 392ms/step - loss: 0.5536 - accuracy: 0.7282 - val_loss: 0.5849 - val_accuracy: 0.6972
CPU times: total: 2h 42min 33s
Wall time: 41min 36s


# LSTM


In [55]:
My_LSTM_Model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=input_dim, output_dim=32, input_length=500),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [56]:
My_LSTM_Model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           342816    
                                                                 
 bidirectional_1 (Bidirectio  (None, 100)              33200     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 376,117
Trainable params: 376,117
Non-trainable params: 0
_________________________________________________________________


In [57]:
loss_function = keras.losses.BinaryCrossentropy(from_logits=False)
My_LSTM_Model.compile(
                 loss=loss_function,
                 metrics=["accuracy"],
                 optimizer='adam'
                 )

In [58]:
print(training_data[0:3, 0:3])
print(training_data.shape)
print(train_labels[10])

[[10711  4207 10711]
 [10711 10711  7496]
 [10711 10711  6912]]
(40000, 500)
[1]


In [59]:
%%time
Hist=My_LSTM_Model.fit(training_data, train_labels, batch_size=12, epochs=5, validation_data=(testing_data, test_labels))

Epoch 1/5
3334/3334 [==============================] - 2175s 648ms/step - loss: 0.5643 - accuracy: 0.7034 - val_loss: 0.5807 - val_accuracy: 0.6936
Epoch 2/5
3334/3334 [==============================] - 1951s 585ms/step - loss: 0.4498 - accuracy: 0.8035 - val_loss: 0.4740 - val_accuracy: 0.7794
Epoch 3/5
3334/3334 [==============================] - 1900s 570ms/step - loss: 0.3775 - accuracy: 0.8396 - val_loss: 0.3382 - val_accuracy: 0.8596
Epoch 4/5
3334/3334 [==============================] - 1883s 565ms/step - loss: 0.3046 - accuracy: 0.8789 - val_loss: 0.3539 - val_accuracy: 0.8608
Epoch 5/5
3334/3334 [==============================] - 1892s 567ms/step - loss: 0.2644 - accuracy: 0.8964 - val_loss: 0.3355 - val_accuracy: 0.8662
CPU times: total: 9h 38min 1s
Wall time: 2h 43min 21s


# CNN


In [61]:
My_CNN_Model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=input_dim, output_dim=32, input_length=500),
  
  tf.keras.layers.Conv1D(50, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  
  tf.keras.layers.Conv1D(40, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  
  tf.keras.layers.Conv1D(30, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  
  tf.keras.layers.Conv1D(30, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  
  tf.keras.layers.Flatten(),
 
  tf.keras.layers.Dense(20),
  tf.keras.layers.Dropout(0.5),
 
  tf.keras.layers.Dense(1, activation="sigmoid"),

  
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [62]:
My_CNN_Model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 32)           342816    
                                                                 
 conv1d (Conv1D)             (None, 498, 50)           4850      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 249, 50)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 247, 40)           6040      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 123, 40)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 121, 30)          

In [63]:
loss_function = keras.losses.BinaryCrossentropy(from_logits=False)
My_CNN_Model.compile(
                 loss=loss_function,
                 metrics=["accuracy"],
                 optimizer='adam'
                 )

In [64]:
print(training_data[0:3, 0:3])
print(training_data.shape)
print(train_labels[10])

[[10711  4207 10711]
 [10711 10711  7496]
 [10711 10711  6912]]
(40000, 500)
[1]


In [65]:
%%time
Hist=My_CNN_Model.fit(training_data, train_labels, batch_size=12, epochs=5, validation_data=(testing_data, test_labels))

Epoch 1/5
3334/3334 [==============================] - 555s 165ms/step - loss: 0.6934 - accuracy: 0.4997 - val_loss: 0.6932 - val_accuracy: 0.4990
Epoch 2/5
3334/3334 [==============================] - 598s 179ms/step - loss: 0.6932 - accuracy: 0.5035 - val_loss: 0.6932 - val_accuracy: 0.5010
Epoch 3/5
3334/3334 [==============================] - 629s 189ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4990
Epoch 4/5
3334/3334 [==============================] - 536s 161ms/step - loss: 0.6932 - accuracy: 0.5001 - val_loss: 0.6931 - val_accuracy: 0.4990
Epoch 5/5
3334/3334 [==============================] - 557s 167ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6932 - val_accuracy: 0.5010
CPU times: total: 3h 23min 17s
Wall time: 47min 55s


In [67]:
print("Evaluate model on test data")
results = My_CNN_Model.evaluate(testing_data, test_labels, batch_size=256)
print("test loss, test acc:", results)

Evaluate model on test data


20/20 [==============================] - 2s 94ms/step - loss: 0.6932 - accuracy: 0.5010
test loss, test acc: [0.6931760907173157, 0.5009999871253967]


Generate a prediction using `model.predict()` and calculate it's shape:

In [68]:
print("Generate a prediction")
prediction = My_CNN_Model.predict(testing_data)
print(prediction)
print("prediction shape:", prediction.shape)
print(type(prediction))
prediction[prediction > .5] = 1
prediction[prediction <= .5] = 0
print(prediction)

Generate a prediction
157/157 [==============================] - 7s 34ms/step
[[0.504932]
 [0.504932]
 [0.504932]
 ...
 [0.504932]
 [0.504932]
 [0.504932]]
prediction shape: (5000, 1)
<class 'numpy.ndarray'>
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [69]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(prediction, test_labels))

[[   0    0]
 [2495 2505]]


# One more example - An ANN used on image data that is Flattened


In [70]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

## Load the CIFAR-10 dataset

In [71]:
(training_data2, train_labels2), (testing_data2, test_labels2) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


## Scale the pixel values to between 0 and 1

In [72]:
training_data2 = training_data2 / 255.0
testing_data2 = testing_data2 / 255.0

## Convert the labels to one-hot encoding

In [73]:
train_labels2 = to_categorical(train_labels2)
test_labels2 = to_categorical(test_labels2)

In [74]:
ANN_Model_Images = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
  tf.keras.layers.Dense(200, activation='relu'), 
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(10, activation="softmax"),  
])

In [75]:
ANN_Model_Images.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_10 (Dense)            (None, 200)               614600    
                                                                 
 dense_11 (Dense)            (None, 150)               30150     
                                                                 
 dense_12 (Dense)            (None, 10)                1510      
                                                                 
Total params: 646,260
Trainable params: 646,260
Non-trainable params: 0
_________________________________________________________________


In [76]:
ANN_Model_Images.compile(
                 loss="categorical_crossentropy",
                 metrics=["accuracy"],
                 optimizer='adam'
                 )

In [77]:
%%time
Hist= ANN_Model_Images.fit(training_data2, train_labels2, epochs=50, validation_data=(testing_data2, test_labels2))

Epoch 1/50
1563/1563 [==============================] - 30s 17ms/step - loss: 1.8601 - accuracy: 0.3287 - val_loss: 1.7122 - val_accuracy: 0.3851
Epoch 2/50
1563/1563 [==============================] - 29s 18ms/step - loss: 1.6742 - accuracy: 0.3992 - val_loss: 1.6167 - val_accuracy: 0.4124
Epoch 3/50
1563/1563 [==============================] - 25s 16ms/step - loss: 1.5954 - accuracy: 0.4274 - val_loss: 1.5727 - val_accuracy: 0.4323
Epoch 4/50
1563/1563 [==============================] - 29s 18ms/step - loss: 1.5512 - accuracy: 0.4451 - val_loss: 1.5428 - val_accuracy: 0.4461
Epoch 5/50
1563/1563 [==============================] - 23s 15ms/step - loss: 1.5186 - accuracy: 0.4585 - val_loss: 1.5605 - val_accuracy: 0.4435
Epoch 6/50
1563/1563 [==============================] - 42s 27ms/step - loss: 1.4969 - accuracy: 0.4639 - val_loss: 1.4981 - val_accuracy: 0.4638
Epoch 7/50
1563/1563 [==============================] - 32s 20ms/step - loss: 1.4738 - accuracy: 0.4731 - val_loss: 1.5250 -